In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time
import os


In [ ]:
# ✅ Hyperparameters
BUFFER_SIZE = 60000
BATCH_SIZE = 256
EPOCHS = 30
NOISE_DIM = 100
NUM_EXAMPLES_TO_GENERATE = 16
SAVE_INTERVAL = 5  # Save image every N epochs
# ✅ Fixed seed for consistent evaluation
seed = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])


In [ ]:
# ✅ Generator
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(7*7*256, use_bias=False, input_shape=(NOISE_DIM,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((7, 7, 256)),
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

In [ ]:
# ✅ Discriminator
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(28, 28, 1)),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

In [ ]:
# ✅ Loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# ✅ Optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:

# ✅ Training step
@tf.function
def train_step(images, generator, discriminator):
    noise = tf.random.normal([tf.shape(images)[0], NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [ ]:
# ✅ Generate and Save Images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions * 127.5 + 127.5).numpy().astype(np.uint8)

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, 0], cmap='gray')
        plt.axis('off')

    plt.tight_layout()
    plt.suptitle(f"Epoch {epoch}", fontsize=12)
    plt.show()

   # plt.tight_layout()
   # os.makedirs('generated', exist_ok=True)
   # plt.savefig(f'generated/image_epoch_{epoch:03d}.png')
   # plt.close()

In [ ]:
# ✅ Training loop
def train(dataset, generator, discriminator):
    for epoch in range(1, EPOCHS + 1):
        start = time.time()
        gen_loss_total = 0
        disc_loss_total = 0
        num_batches = 0

        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch, generator, discriminator)
            gen_loss_total += gen_loss
            disc_loss_total += disc_loss
            num_batches += 1

        print(f'Epoch {epoch:3d} | Gen Loss: {gen_loss_total/num_batches:.4f} | Disc Loss: {disc_loss_total/num_batches:.4f} | Time: {time.time() - start:.2f}s')

        if epoch % SAVE_INTERVAL == 0 or epoch == 1:
            generate_and_save_images(generator, epoch, seed)

In [ ]:
# ✅ Load and preprocess MNIST
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = (train_images - 127.5) / 127.5
train_images = train_images[..., tf.newaxis]  # Add channel dimension
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


In [ ]:

# ✅ Build and train models
generator = make_generator_model()
discriminator = make_discriminator_model()

train(train_dataset, generator, discriminator)


In [ ]:
train(train_dataset, generator, discriminator)